In [1]:
%cd ..

d:\nlp\assignment1


d:\nlp\nlpenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
from Task1.task1 import WordPieceTokenizer
from Task2.task2 import Word2VecModel


# Neural LM Dataset class
class NeuralLMDataset(Dataset):
    def __init__(self, tokenizer, word2vec, context_size=3):
        self.tokenizer = tokenizer
        self.word2vec = word2vec
        self.context_size = context_size
        self.data = self.preprocess_data()
        # self.pad_token = '[PAD]'
        # self.context_size = context_size
        # self.word2idx = None
        # self.corpus = None
        # self.data = []
        # for sentence in corpus:
        #     words = sentence.split()
        #     for i in range(context_size, len(words) - 1):
        #         context = words[i - context_size:i]
        #         target = words[i]
        #         self.data.append((context, target))
        # self.word2idx = word2idx

    def tokenize_txt_file(self, input_file: str, output_file: str) -> None:
        """Tokenize sentences from input TXT file and save results as JSON"""
        # Reading the input text file
        with open(input_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        results = {}
        for idx, line in enumerate(lines):
            sentence = line.strip()  # Remove leading/trailing whitespaces or newlines
            tokens = self.tokenizer.tokenize(sentence)  # Tokenize the sentence using your tokenization method
            results[str(idx)] = tokens  # Store tokens with index as the key (starting from 0)
        # Saving the results to an output JSON file
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2)

    def preprocess_data(self):
        self.tokenizer.construct_vocabulary("corpus.txt", vocab_size=100)
        self.tokenize_txt_file("corpus.txt", "tokenized_corpus.json")

        corpus = None
        with open('tokenized_corpus.json', 'r', encoding='utf-8') as f:
            # Load the JSON data
            tokenized_corpus = json.load(f)
            
            # Convert the dictionary into a list of sentences (list of tokenized words)
            corpus = [tokens for tokens in tokenized_corpus.values()]
            
        
        self.tokenized_sentences = corpus
        # updates the word to index mapping
        self.word2idx = {word: idx for idx, word in enumerate(self.tokenizer.vocab)}
        # updates the reverse index to word mapping
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        # indexed_sentences = [[self.word2vec.embeddings(torch.tensor(self.word2idx[token], dtype=torch.long)).detach().numpy() for token in sent] for sent in self.tokenized_sentences]
        data = []

        for sent in self.tokenized_sentences:
            if len(sent) > self.context_size:
                for i in range(len(sent) - self.context_size):
                    # context = sent[i:i+self.context_size]
                    context = [self.word2vec.embeddings(torch.tensor(self.word2idx[token], dtype=torch.long)).detach().numpy() for token in sent[i:i+self.context_size]]
                    target = sent[i+self.context_size]
                    if target in self.word2idx:
                        target_idx = self.word2idx[target]
                    else:
                        target_idx = self.word2idx['[UNK]']
                    data.append((context, target_idx))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        context_tensor = torch.tensor(np.concatenate(context).flatten(), dtype=torch.float32)
        target_tensor = torch.tensor(target, dtype=torch.long)
        return context_tensor, target_tensor

# Define Neural Language Model Variations
class NeuralLM1(nn.Module):
    def __init__(self, input_dim, hidden_dim, vocab_size):
        super(NeuralLM1, self).__init__()
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        # x = self.embedding(x).view(x.shape[0], -1)
        x = self.relu(self.fc(x))
        return self.out(x)

class NeuralLM2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(NeuralLM2, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, hidden_dim)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x).view(x.shape[0], -1)
        x = self.tanh(self.fc1(x))
        return self.fc2(x)

class NeuralLM3(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(NeuralLM3, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, hidden_dim)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x).view(x.shape[0], -1)
        x = self.relu(self.batch_norm(self.fc1(x)))
        x = self.dropout(x)
        return self.fc2(x)

# Training function with loss tracking
def train(model, train_dataloader, val_dataloader, epochs=100, lr=0.01):
    model.to("cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_losses, val_losses = [], []
    train_acc, val_acc = [], []

    for epoch in range(epochs):
        #training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for context, target in train_dataloader:
            optimizer.zero_grad() #zero the gradients
            #forward pass
            output = model(context)
            loss = criterion(output, target)
            train_loss += loss.item()
            #backward pass and optimize
            loss.backward()
            optimizer.step()
            #compute accuracy
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)
        #calculate average training loss
        train_losses.append(train_loss / len(train_dataloader))
        train_acc.append(100 * correct / total)

        #validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for context, target in val_dataloader:
                output = model(context)
                loss = criterion(output, target)
                val_loss += loss.item()
                #compute accuracy
                _, predicted = torch.max(output, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)
        #calculate average validation loss
        val_losses.append(val_loss / len(val_dataloader))
        val_acc.append(100 * correct / total)

        #print losses for each epoch
        print(f"----- Epoch {epoch + 1}/{epochs} -----")
        # print(f"Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")
        print(f"Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_acc[-1]:.2f}%")
        print(f"Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_acc[-1]:.2f}%")

    return train_losses, train_acc, val_losses, val_acc #return losses

# Plot loss function
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs Epochs')
    plt.legend()
    plt.show()

# Accuracy and Perplexity computation
def compute_accuracy(model, dataloader):
    correct, total = 0, 0
    with torch.no_grad():
        for context, target in dataloader:
            output = model(context)
            predictions = torch.argmax(output, dim=1)
            correct += (predictions == target).sum().item()
            total += target.size(0)
    return correct / total

def compute_perplexity(loss):
    return np.exp(loss)

In [3]:
%cd task3

d:\nlp\assignment1\task3


d:\nlp\nlpenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
def load_model(final_model_dir, model_class):
    model_path = r'D:\nlp\assignment1\Task2\final_model\final_model.pt'
    checkpoint = torch.load(model_path)
    
    model = model_class(vocab_size=checkpoint['vocab_size'], embedding_dim=checkpoint['embedding_dim'])
    model.load_state_dict(checkpoint['model_state_dict'])
    
    val_loss = checkpoint['val_loss']
    val_accuracy = checkpoint['val_accuracy']
    
    return model, val_loss, val_accuracy

def load_vocabulary(final_model_dir):
    vocab_path = r'D:\nlp\assignment1\Task2\final_model\vocabulary.json'
    with open(vocab_path, 'r') as f:
        vocab_data = json.load(f)
    return vocab_data['word2idx'], vocab_data['idx2word']


# Example usage:
word2vec, val_loss, val_accuracy = load_model('final_model', Word2VecModel)
word2idx, idx2word = load_vocabulary('final_model')
tokenizer = WordPieceTokenizer()
dataset = NeuralLMDataset(tokenizer, word2vec)

WINDOW_SIZE = 2
BATCH_SIZE = 128
NUM_EPOCHS = 10
LEARNING_RATE = 0.005
TRAIN_SPLIT = 0.8

# Split dataset into training and validation
train_size = int(TRAIN_SPLIT * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


model = NeuralLM1(input_dim=300, hidden_dim=256, vocab_size=100)

In [5]:
train(model, train_loader, val_loader, epochs=NUM_EPOCHS, lr=LEARNING_RATE)

----- Epoch 1/10 -----
Train Loss: 2.4937, Train Accuracy: 33.28%
Val Loss: 2.3312, Val Accuracy: 36.56%
----- Epoch 2/10 -----
Train Loss: 2.2759, Train Accuracy: 37.42%
Val Loss: 2.2691, Val Accuracy: 37.53%
----- Epoch 3/10 -----
Train Loss: 2.2135, Train Accuracy: 38.58%
Val Loss: 2.2483, Val Accuracy: 38.28%
----- Epoch 4/10 -----
Train Loss: 2.1832, Train Accuracy: 39.14%
Val Loss: 2.2304, Val Accuracy: 39.24%
----- Epoch 5/10 -----
Train Loss: 2.1576, Train Accuracy: 39.72%
Val Loss: 2.2021, Val Accuracy: 39.41%
----- Epoch 6/10 -----
Train Loss: 2.1390, Train Accuracy: 40.02%
Val Loss: 2.2245, Val Accuracy: 39.16%
----- Epoch 7/10 -----
Train Loss: 2.1262, Train Accuracy: 40.47%
Val Loss: 2.1976, Val Accuracy: 39.95%
----- Epoch 8/10 -----
Train Loss: 2.1170, Train Accuracy: 40.51%
Val Loss: 2.1886, Val Accuracy: 39.86%
----- Epoch 9/10 -----
Train Loss: 2.1062, Train Accuracy: 40.68%
Val Loss: 2.2010, Val Accuracy: 39.95%
----- Epoch 10/10 -----
Train Loss: 2.1032, Train Accur

([2.493681924516,
  2.2759403121610147,
  2.2135160806036036,
  2.183190859634106,
  2.1575845014654105,
  2.139018481292144,
  2.1262141663577796,
  2.117036268875443,
  2.1061677941277077,
  2.1031842189061396],
 [33.277099156487346,
  37.42396135942039,
  38.57897868468027,
  39.13828707430611,
  39.719995799936996,
  40.02450036750551,
  40.468307024605366,
  40.5138077071156,
  40.67971019565294,
  40.718910783661755],
 [2.3311901339462824,
  2.2691075780561993,
  2.248342454433441,
  2.2304088030542646,
  2.202116648214204,
  2.224470841884613,
  2.197626953465598,
  2.188559335895947,
  2.2009861043521335,
  2.1930630160229545],
 [36.56269250154001,
  37.53150025200202,
  38.27910623284986,
  39.24231393851151,
  39.41311530492244,
  39.161113288906314,
  39.953519628157025,
  39.86111888895111,
  39.945119560956485,
  40.07112056896455])

In [13]:
sentence = "The weather is"
model.eval()
words = tokenizer.tokenize(sentence)
context = [dataset.word2vec.embeddings(torch.tensor(dataset.word2idx[token], dtype=torch.long)).detach().numpy() for token in words][-3:]
context = torch.tensor(np.concatenate(context).flatten(), dtype=torch.float32).unsqueeze(0)  # Add batch dim

In [17]:
print(words)

['th', '##e', 'w', '##e', '##a', '##t', '##h', '##e', '##r', 'i', '##s']


In [16]:
predicted_words = []
for _ in range(3):
    with torch.no_grad():
        output = model(context)  # Predict next word
        next_word_idx = torch.argmax(output, dim=1).item()  # Get most probable word
        next_word = dataset.idx2word.get(next_word_idx, "<UNK>")  # Convert index to word
        predicted_words.append(next_word)

        # Update context by adding new word embedding
        if next_word in word2idx:
            next_embedding = dataset.word2vec.embeddings(torch.tensor(word2idx[next_word], dtype=torch.long)).detach()
            next_embedding = next_embedding.unsqueeze(0).unsqueeze(1)  # Reshape for batch
            context = torch.cat([context[:, 1:, :], next_embedding], dim=1)

print(" ".join(predicted_words))

IndexError: too many indices for tensor of dimension 2

In [ ]:
def predict_next_words(model, sentence, tokenizer, word2vec, vocab, num_words=3):
    model.eval()
    words = tokenizer.tokenize(sentence)
    
    # Convert to embeddings
    context = [word2vec[word] for word in words if word in word2vec][-3:]  # Use last 3 tokens
    context = torch.tensor(context, dtype=torch.float32).unsqueeze(0)  # Add batch dim

    predicted_words = []
    for _ in range(num_words):
        with torch.no_grad():
            output = model(context)  # Predict next word
            next_word_idx = torch.argmax(output, dim=1).item()  # Get most probable word
            next_word = vocab[next_word_idx]  # Convert index to word
            predicted_words.append(next_word)

            # Update context by adding new word embedding
            if next_word in word2vec:
                next_embedding = word2vec[next_word]
                context = torch.cat([context[:, 1:, :], torch.tensor(next_embedding).unsqueeze(0).unsqueeze(1)], dim=1)

    return " ".join(predicted_words)

In [ ]:
sentence = "The weather is"
predicted = predict_next_words(model, sentence, tokenizer, word2vec, vocab)
print(f"Predicted words: {predicted}")

In [ ]:
# Prepare data for training
dataset = NeuralLMDataset(corpus, text_processor.word2idx)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Train models
embedding_dim = 10
hidden_dim = 20

models = [NeuralLM1, NeuralLM2, NeuralLM3]
losses = {}
accuracies = {}
perplexities = {}

for i, model_class in enumerate(models):
    model = model_class(len(text_processor.vocab), embedding_dim, hidden_dim)
    loss = train(model, dataloader)
    losses[f'NeuralLM{i+1}'] = loss
    acc = compute_accuracy(model, dataloader)
    accuracies[f'NeuralLM{i+1}'] = acc
    perplexities[f'NeuralLM{i+1}'] = compute_perplexity(loss[-1])
    plot_losses(loss, f'NeuralLM{i+1} Training Loss')
    torch.save(model.state_dict(), f"neural_lm{i+1}.pth")
    print(f"NeuralLM{i+1} - Accuracy: {acc}, Perplexity: {perplexities[f'NeuralLM{i+1}']}")

# Predict next 3 tokens from test.txt
def predict_next_tokens(model, sentence, n=3):
    words = sentence.split()
    context = torch.tensor([text_processor.word2idx[word] for word in words[-2:]], dtype=torch.long)
    predictions = []
    for _ in range(n):
        output = model(context.unsqueeze(0))
        next_word_idx = torch.argmax(output, dim=1).item()
        next_word = text_processor.idx2word[next_word_idx]
        predictions.append(next_word)
        context = torch.cat((context[1:], torch.tensor([next_word_idx])))
    return predictions

print("Model checkpoints saved!")